<a href="https://colab.research.google.com/github/wandb/examples/blob/master/colabs/pytorch-lightning/Transfer_Learning_Using_PyTorch_Lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{pytorch-lightning-transfer-learning-colab} -->

<img src="https://wandb.me/logo-im-png" width="400" alt="Weights & Biases" />

<!--- @wandbcode{pytorch-lightning-transfer-learning-colab} -->

# Transfer Learning Using PyTorch Lightning ⚡️

In this colab, we will extend the pipeline [here](https://colab.research.google.com/github/wandb/examples/blob/master/colabs/pytorch-lightning/Image_Classification_using_PyTorch_Lightning.ipynb) to perform transfer learning with PyTorch Lightning.

Transfer Learning is a technique where the knowledge learned while training a model for "task" A and can be used for "task" B. Here A and B can be the same deep learning tasks but on a different dataset.


## Setting up PyTorch Lightning and W&B

For this tutorial, we need PyTorch Lightning and Weights and Biases.

In [ ]:
!pip install -U wandb pytorch-lightning -qqq

You're gonna need these imports.

In [ ]:
import os

import pytorch_lightning as pl
# your favorite machine learning tracking tool
from pytorch_lightning.loggers import WandbLogger

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import random_split, DataLoader

from torchmetrics import Accuracy

from torchvision import transforms
from torchvision.datasets import Flowers102
from torchvision.datasets.utils import download_url
import torchvision.models as models


import wandb

Now you'll need to login to you wandb account.


In [ ]:
wandb.login()

## The Dataset 💿

We will be using the Oxford 102 Flower dataset to train our image classifier. The dataset consists of 102 flower categories. The flowers were chosen to be flowers commonly occurring in the United Kingdom. Each class consists of between 40 and 258 images.

The images have large scale, pose and light variations. In addition, there are categories that have large variations within the category, and several very similar categories.

In [ ]:
class Flowers102DataModule(pl.LightningDataModule):
    def __init__(self, batch_size, data_dir: str = './'):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size

        # Augmentation policy for training set
        self.augmentation = transforms.Compose([
              transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
              transforms.RandomRotation(degrees=15),
              transforms.RandomHorizontalFlip(),
              transforms.CenterCrop(size=224),
              transforms.ToTensor(),
              transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
        ])
        # Preprocessing steps applied to validation and test set.
        self.transform = transforms.Compose([
              transforms.Resize(size=256),
              transforms.CenterCrop(size=224),
              transforms.ToTensor(),
              transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
        ])

        self.num_classes = 196

    def prepare_data(self):
        pass

    def setup(self, stage=None):
        # build dataset
        dataset = Flowers102(root=self.data_dir, download=True, split="train")
        # split dataset
        self.train, self.val = random_split(dataset, [816, 204])

        self.test = Flowers102(root=self.data_dir, download=True, split="test")

        self.test = random_split(self.test, [len(self.test)])[0]

        self.train.dataset.transform = self.augmentation
        self.val.dataset.transform = self.transform
        self.test.dataset.transform = self.transform

    def train_dataloader(self):
        return DataLoader(self.train, batch_size=self.batch_size, shuffle=True, num_workers=2)

    def val_dataloader(self):
        return DataLoader(self.val, batch_size=self.batch_size, num_workers=2)

    def test_dataloader(self):
        return DataLoader(self.test, batch_size=self.batch_size, num_workers=2)

## LightingModule - Define the System

Let us look at the model definition to see how transfer learning can be used with PyTorch Lightning.
In the `LitModel` class, we can use the pre-trained model provided by Torchvision as a feature extractor for our classification model. Here we are using ResNet-18. A list of pre-trained models provided by PyTorch Lightning can be found [here](https://pytorch.org/vision/stable/models.html#classification).
- When `pretrained=True`, we use the pre-trained weights; otherwise, the weights are initialized randomly.
- If `.eval()` is used, then the layers are frozen.
- A single `Linear` layer is used as the output layer. We can have multiple layers stacked over the `feature_extractor`.

Setting the `transfer` argument to `True` will enable transfer learning.

In [ ]:
class LitModel(pl.LightningModule):
    def __init__(self, input_shape, num_classes, learning_rate=2e-4, transfer=False):
        super().__init__()

        # log hyperparameters
        self.save_hyperparameters()
        self.learning_rate = learning_rate
        self.dim = input_shape
        self.num_classes = num_classes

        # transfer learning if pretrained=True
        self.feature_extractor = models.resnet18(pretrained=transfer)

        if transfer:
            # layers are frozen by using eval()
            self.feature_extractor.eval()
            # freeze params
            for param in self.feature_extractor.parameters():
                param.requires_grad = False

        n_sizes = self._get_conv_output(input_shape)

        self.dropout = nn.Dropout(0.5)
        self.classifier = nn.Linear(n_sizes, num_classes)

        self.criterion = nn.CrossEntropyLoss()
        self.accuracy = Accuracy(task="multiclass", num_classes=num_classes)
        self.test_step_outputs = []

    # returns the size of the output tensor going into the Linear layer from the conv block.
    def _get_conv_output(self, shape):
        batch_size = 1
        tmp_input = torch.autograd.Variable(torch.rand(batch_size, *shape))

        output_feat = self._forward_features(tmp_input)
        n_size = output_feat.data.view(batch_size, -1).size(1)
        return n_size

    # returns the feature tensor from the conv block
    def _forward_features(self, x):
        x = self.feature_extractor(x)
        return x

    # will be used during inference
    def forward(self, x):
       x = self._forward_features(x)
       x = x.view(x.size(0), -1)
       x = self.dropout(x)
       x = self.classifier(x)

       return x

    def training_step(self, batch):
        batch, gt = batch[0], batch[1]
        out = self.forward(batch)
        loss = self.criterion(out, gt)

        acc = self.accuracy(out, gt)

        self.log("train/loss", loss)
        self.log("train/acc", acc)

        return loss

    def validation_step(self, batch, batch_idx):
        batch, gt = batch[0], batch[1]
        out = self.forward(batch)
        loss = self.criterion(out, gt)

        self.log("val/loss", loss)

        acc = self.accuracy(out, gt)
        self.log("val/acc", acc)

        return loss

    def test_step(self, batch, batch_idx):
        batch, gt = batch[0], batch[1]
        out = self.forward(batch)
        loss = self.criterion(out, gt)
        self.test_step_outputs.append({"loss": loss, "outputs": out, "gt": gt})
        return out

    def on_test_epoch_end(self):
        outputs = self.test_step_outputs
        loss = torch.stack([x['loss'] for x in outputs]).mean()
        output = torch.cat([x['outputs'] for x in outputs], dim=0)

        gts = torch.cat([x['gt'] for x in outputs], dim=0)

        self.log("test/loss", loss)
        acc = self.accuracy(output, gts)
        self.log("test/acc", acc)

        self.test_gts = gts
        self.test_output = output
        self.test_step_outputs.clear()

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)

## Train your Model 🏋️‍♂️

To train the model, we instantiate the `StanfordCarsDataModule` and the `LitModel` along with the PyTorch Lightning Trainer. To the `Trainer`, we will pass the `WandbLogger` as the logger to use W&B to track the metrics during model training!.

### Model logging checkpointing

We will use the `ModelCheckpoint` callback to save the best model. We can easily save our model alongside our wandb run as a model artifact by passing `log_model=True` to our `WandbLogger`. This will not only facilitate easy reproducibility of our models from the experiments but also allow us to reuse the model from [Weights & Biases Models](https://docs.wandb.ai/guides/models) during inference.

In [ ]:
dm = Flowers102DataModule(batch_size=32)
model = LitModel((3, 300, 300), 196, transfer=True)
checkpoint_callback = pl.callbacks.ModelCheckpoint(monitor="val/acc", mode="max", save_top_k=3)
wandb_logger = WandbLogger(project="TransferLearning", log_model="all")
trainer = pl.Trainer(
    max_epochs=25,
    accelerator="gpu",
    log_every_n_steps=26,
    callbacks=[checkpoint_callback],
    logger=wandb_logger
    )

We are good to go! Let's train our model!

In [ ]:
trainer.fit(model, dm)

## Model Evaluation

### Load the best model using Weights & Biases Artifacts

Although we ran our training for 15 epochs, we want to test against the best model. We can use the model artifact stored in wandb using the `wandb_logger` to retrieve the checkpoints for the best model and reload the model from it.

In [ ]:
artifact_path = f'{wandb.run.entity}/{wandb.run.project}/model-{wandb.run.id}:best'
artifact = wandb_logger.use_artifact(artifact_path, artifact_type='model',)
artifact_dir = artifact.download()
model = LitModel.load_from_checkpoint(f"{artifact_dir}/model.ckpt")

Now that we have our best model, let's see how it performs on the test set by evaluating the model on the test set.

In [ ]:
trainer.test(model, dm)

Let's close our W&B run, so we call `wandb.finish()`.

In [ ]:
display(wandb.run)
wandb.finish()

The workspace generated to compare training the model from scratch vs using transfer learning is available [here](https://wandb.ai/parambharat/TransferLearning)

## Conclusion

We encourage you to play with the code and train an image classifier with a dataset of your choice from scratch and using transfer learning.


To learn more about transfer learning check out these resources:
- [Gotchas of transfer learning for image classification](https://docs.google.com/presentation/d/1s29WOQoQvBD5KoPUzE5TPcavjqno8ZgnZaSljHGGHVU/edit?usp=sharing) by Sayak Paul.
- [Transfer Learning with Keras and Deep Learning by PyImageSearch.](https://www.pyimagesearch.com/2019/05/20/transfer-learning-with-keras-and-deep-learning/)
- [Transfer Learning - Machine Learning's Next Frontier](https://ruder.io/transfer-learning/) by Sebastian Ruder.





